In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

In [13]:
class FeedForwardNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_layers = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256,10)
        )
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, input_data):
        flattened_data = self.flatten(input_data)
        logits = self.dense_layers(flattened_data)
        predictions = self.softmax(logits)
        return predictions
        

In [14]:
def donwload_mnist_dataset():
    train_data = datasets.MNIST(
        root = "data",
        download = True,
        train = True,
        transform=ToTensor()
    )
    validation_data = datasets.MNIST(
        root = "data",
        download = True,
        train = False,
        transform=ToTensor()
    )
    return train_data, validation_data


In [15]:
#download dataset
train_data, _ = donwload_mnist_dataset()
print("MNIST dataset downloaded")

MNIST dataset downloaded


In [16]:
def train_one_epoch(model,data_loader,loss_fn,optimizer, device):
    for inputs,targets in data_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        #calculate loss
        predictions = model(inputs)
        loss = loss_fn(predictions,targets)
        
        #backpropagate loss and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"loss: {loss.item()}")
    
def train(model,data_loader,loss_fn,optimizer, device,epochs):
    for i in range(epochs):
        print(f"Epoch {i+1}")
        train_one_epoch(model,data_loader,loss_fn,optimizer, device)
        print("------------------")
    print("Training is done.train_data_loader")
    

In [17]:
# create dataloader for dataset
train_data_loader = DataLoader(train_data,batch_size=BATCH_SIZE)

In [18]:
#build model
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"using {device} device")
feed_forward_net = FeedForwardNet().to(device)

using cpu device


In [19]:
## Lost function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(feed_forward_net.parameters(),lr = LEARNING_RATE)

##Train Model
train(feed_forward_net,train_data_loader, loss_fn, optimizer, device, EPOCHS)

torch.save(feed_forward_net.state_dict(),"feedforwardnet.pth")
print("model trained and saved")

Epoch 1
loss: 1.5115442276000977
------------------
Epoch 2
loss: 1.4973483085632324
------------------
Epoch 3
loss: 1.4836505651474
------------------
Epoch 4
loss: 1.480048656463623
------------------
Epoch 5
loss: 1.4768809080123901
------------------
Epoch 6
loss: 1.4745985269546509
------------------
Epoch 7
loss: 1.472458004951477
------------------
Epoch 8
loss: 1.4732789993286133
------------------
Epoch 9
loss: 1.472510814666748
------------------
Epoch 10
loss: 1.4732288122177124
------------------
Training is done.train_data_loader
model trained and saved
